<a href="https://colab.research.google.com/github/jackflaggg/Fit-file/blob/master/labOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install fitdecode

In [49]:
!ls

sample_data  travelTwo.fit


In [39]:
f1 = open('/content/travelTwo.fit', 'r')

In [40]:
record = ['latitude', 'longitude', 'laltitude', 'timestamp', 'speed', 'distance']

In [41]:
import fitdecode
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Union, Optional,Tuple

In [42]:
def get_fit_point_data(frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:

  data: Dict[str, Union[float, int, str, datetime]] = {}

  if not (frame.has_field('position_lat') and frame.has_field('position_long')):
    return None
  else:
    data['latitude'] = frame.get_value('position_lat') / ((2**32) / 360)
    data['longitude'] = frame.get_value('position_long') / ((2**32) / 360)

  for field in record[2:]:
    if frame.has_field(field):
      data[field] = frame.get_value(field)

  return data


In [43]:
def get_fit_other_data(col, frame: fitdecode.records.FitDataMessage) -> Optional[Dict[str, Union[float, int, str, datetime]]]:
  data: Dict[str, Union[float, int, str, datetime]] = {}

  for field in col:
    if frame.has_field(field):
      data[field] = frame.get_value(field)
  return data


In [44]:
record_data = []
with fitdecode.FitReader('/content/travelTwo.fit') as fit_file:
   for frame in fit_file:
    if isinstance(frame, fitdecode.records.FitDataMessage):
      if frame.name == 'record':
        single_point_data = get_fit_point_data(frame)
        if single_point_data is not None:
          record_data.append(single_point_data)


record_df = pd.DataFrame(record_data, columns=record)
#record_df


In [45]:
from pyproj import CRS
import geopandas as gpd
from shapely.geometry import LineString

In [46]:
line = gpd.GeoDataFrame(crs = CRS('EPSG:4326'), geometry=[LineString(zip(record_df.longitude, record_df.latitude))])
line_centroid = line.centroid
line_bounds = line.bounds


<ipython-input-46-c46bb885f153>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  line_centroid = line.centroid


In [47]:
import folium
m = folium.Map([line_centroid.y, line_centroid.x], tiles='cartodb positron')
folium.GeoJson(line).add_to(m)
folium.FitBounds([[line_bounds.miny[0], line_bounds.minx[0]],[line_bounds.maxy[0], line_bounds.maxx[0]]]).add_to(m)
m


In [48]:
import altair as alt

# Сначала добавьте столбец 'distance' в DataFrame
record_df['cumulative_distance'] = record_df['distance'].cumsum()

# Построение графика относительно расстояния и времени
chart = alt.Chart(record_df).mark_line().encode(
    x='cumulative_distance',
    y='timestamp:T'
).properties(width=800, height=600)

# Отобразить график
chart

alt.Chart(...)